# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import load_model

In [2]:
import numpy as np
import os

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [83]:
model = tf.keras.models.Sequential()

### Step 1 - Convolution

In [84]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3], kernel_regularizer=regularizers.l2(0.001)))

### Step 2 - Pooling

In [85]:
model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [86]:
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [87]:
model.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [88]:
model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))

### Step 5 - Output Layer

In [89]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [90]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


### Training the CNN on the Training set and evaluating it on the Test set

In [91]:
history = model.fit(
    x=training_set,
    validation_data=test_set,
    epochs=30,
)
model.save('cnn_model.keras')


Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - accuracy: 0.5708 - loss: 0.8720 - val_accuracy: 0.6040 - val_loss: 0.7550
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.6674 - loss: 0.7185 - val_accuracy: 0.6700 - val_loss: 0.6830
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - accuracy: 0.6912 - loss: 0.6682 - val_accuracy: 0.7050 - val_loss: 0.6509
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.7098 - loss: 0.6310 - val_accuracy: 0.7050 - val_loss: 0.6384
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - accuracy: 0.7407 - loss: 0.5965 - val_accuracy: 0.6790 - val_loss: 0.6691
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.7428 - loss: 0.5840 - val_accuracy: 0.7165 - val_loss: 0.6156
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - accuracy: 0.7649 - loss: 0.5598 - val_accuracy: 0.7575 - val_loss: 0.5588
Epoch 8/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.7649 - loss: 0.5565 - 

## Part 4 - Making a single prediction

In [7]:
def load_and_preprocess_image(image_path):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image_array = image.img_to_array(test_image)
    test_image_array = np.expand_dims(test_image_array, axis=0)
    test_image_array = test_image_array / 255.0
    return test_image_array

def predict_image(model_name, image_array, class_indices):
    print("Making predictions...")
    result = model_name.predict(image_array)
    print("Prediction raw output:", result)
    print("Class Indices:", class_indices)
    
    if result > 0.5:
        prediction = 'dog'
    else:
        prediction = 'cat'
    print(f"The model predicts this image is a: {prediction}. \n")
    
model = load_model('cnn_model.keras')

In [11]:
print("Image: cat_or_dog_1.jpg is a dog.")
single_image_path_1 = 'dataset/single_prediction/cat_or_dog_1.jpg'
test_image_array_1 = load_and_preprocess_image(single_image_path_1)
predict_image(model, test_image_array_1, training_set.class_indices)

print("Image: cat_or_dog_2.jpg is a cat.")
single_image_path_2 = 'dataset/single_prediction/cat_or_dog_2.jpg'
test_image_array_2 = load_and_preprocess_image(single_image_path_2)
predict_image(model, test_image_array_2, training_set.class_indices)

print("Image: cat_or_dog_3.jpg is a dog.")
single_image_path_3 = 'dataset/single_prediction/cat_or_dog_3.jpg'
test_image_array_3 = load_and_preprocess_image(single_image_path_3)
predict_image(model, test_image_array_3, training_set.class_indices)

print("Image: cat_or_dog_4.jpg is a cat.")
single_image_path_4 = 'dataset/single_prediction/cat_or_dog_4.jpg'
test_image_array_4 = load_and_preprocess_image(single_image_path_4)
predict_image(model, test_image_array_4, training_set.class_indices)

Image: cat_or_dog_1.jpg is a dog.
Making predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction raw output: [[0.9947713]]
Class Indices: {'cats': 0, 'dogs': 1}
The model predicts this image is a: dog. 

Image: cat_or_dog_2.jpg is a cat.
Making predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction raw output: [[0.0724668]]
Class Indices: {'cats': 0, 'dogs': 1}
The model predicts this image is a: cat. 

Image: cat_or_dog_3.jpg is a dog.
Making predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction raw output: [[0.90196526]]
Class Indices: {'cats': 0, 'dogs': 1}
The model predicts this image is a: dog. 

Image: cat_or_dog_4.jpg is a cat.
Making predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction raw output: [[0.13769597]]
Class Indices: {'cats': 0, 'dogs': 1}
The model predicts this image is a: cat. 



## 1. Github link：
https://github.com/Jieyi-Wei/CNN_for_Image_Classification_Project
## 2. Medium link
preparing...